##Acessando bancos de dados NCBI com Biopython

Extraído de https://github.com/lanadominkovic/12-days-of-biopython

Biopython fornece uma interface para Entrez, o sistema de recuperação de dados disponibilizado pelo NCBI. Entrez também pode ser usado por meio de navegador da web: https://www.ncbi.nlm.nih.gov/search/

DICAS:

- Especifique um endereço de e-mail com sua consulta

- Evite um grande número de solicitações (100 ou mais) durante os horários de pico (entre 9h00 e 17h00, horário do leste dos EUA, em dias úteis)

- Não faça mais de três consultas por segundo (o Biopython cuidará disso para você)

In [ ]:
# Instalando o Biopython
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.8 MB/s eta 0:00:00


In [ ]:
# Importando bibliotecas
from Bio import Entrez, SeqIO

In [ ]:
Entrez.email = "cerri@icmc.usp.br"

In [ ]:
#EInfo: obtenha uma lista de todos os nomes de bancos de dados acessíveis através do Entrez
handle = Entrez.einfo()
rec = Entrez.read(handle)
handle.close()
print(rec.keys())

dict_keys(['DbList'])


In [ ]:
rec['DbList']

['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'medgen', 'mesh', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'proteinclusters', 'pcassay', 'protfam', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']

Agora, tentaremos encontrar o gene transportador de resistência à cloroquina (CRT) (KM288867) no Plasmodium falciparum (o parasita que causa a forma mais mortal de malária) no banco de dados de nucleotídeos:

ESearch: Pesquisando nos bancos de dados Entrez

Observe que a pesquisa padrão limitará o número de registroz a 20, então, se tivermos mais, podemos substituir retmax para a quantidade desejada de registros.

In [ ]:
# Wait for a few seconds before retrying the search
import time
time.sleep(5)

In [ ]:
handle = Entrez.esearch(db="nucleotide",
                        term='CRT[Gene Name] AND "Plasmodium falciparum"[Organism] AND "chloroquine resistance transporter"',
                        retmax="10000")
rec_list = Entrez.read(handle)
handle.close()
rec_list['Count']

'3212'

In [ ]:
len(rec_list['IdList'])

3212

In [ ]:
rec_list['IdList']

['3052870581', '3052870578', '3052870575', '3052870572', '3052870569', '3052870566', '3052870563', '3052870560', '3052870557', '3052870554', '3052870551', '3052870548', '3052870545', '3052870542', '3052870539', '3052870536', '3052870533', '3052870530', '3052870527', '3052870524', '3052870521', '3052870518', '3052870515', '3052870512', '3052870509', '3052870506', '3052870503', '3052870500', '3052870497', '3052870494', '3052870491', '3052870488', '3052870485', '3052870482', '3052870479', '3052870476', '3052870473', '3052870470', '3052870467', '3052870464', '3052870461', '3052870458', '3052870455', '3052870452', '3052870449', '3052870446', '3052870443', '3052870440', '3052870437', '3052870434', '3052870431', '3052870428', '3052870425', '3052870422', '3052870419', '3052870416', '3052870413', '3052870410', '3052870407', '3052870404', '3052870401', '3052870398', '3052870395', '3052870392', '3052870389', '3052870386', '3052870383', '3052870380', '3052870377', '3052870374', '3052870371', '3052

Agora temos os IDs de todos os registros, mas ainda precisamos recuperá-los corretamente.

EFetch: Baixando registros completos do Entrez

Solicitar um formato de arquivo específico do Entrez usando Bio.Entrez.efetch() requer a especificação dos argumentos opcionais rettype e/ou retmode. As diferentes combinações são descritas para cada tipo de banco de dados nas páginas vinculadas à página da web do NCBI efetch - https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch

- rettype - tipo de retorno

- gb == GenBank

- retmax - Número total de registros do conjunto de entrada a serem recuperados, até um máximo de 10.000

In [ ]:
id_list = rec_list['IdList']
handle = Entrez.efetch(db='nucleotide', id=id_list, rettype='gb')
recs = list(SeqIO.parse(handle, 'gb'))
handle.close()

Note que convertemos um iterador (o resultado de SeqIO.parse) para uma lista. A vantagem de fazer isso é que podemos usar o resultado quantas vezes quisermos (por exemplo, iterar muitas vezes), sem repetir a consulta no servidor.

In [ ]:
recs

[SeqRecord(seq=Seq('TATATTTTAAGTATTATTTATTTAAGTGTATGTGTAATGAATAAAATTTTTGCT...AAA'), id='PX229102.1', name='PX229102', description='Plasmodium falciparum isolate MMN-2847 chloroquine resistance transporter (crt) gene, partial cds', dbxrefs=[]),
 SeqRecord(seq=Seq('TATATTTTAAGTATTATTTATTTAAGTGTATGTGTAATGAATAAAATTTTTGCT...AAA'), id='PX229101.1', name='PX229101', description='Plasmodium falciparum isolate MMN-2748 chloroquine resistance transporter (crt) gene, partial cds', dbxrefs=[]),
 SeqRecord(seq=Seq('TATATTTTAAGTATTATTTATTTAAGTGTATGTGTAATGAATAAAATTTTTGCT...AAA'), id='PX229100.1', name='PX229100', description='Plasmodium falciparum isolate MMN-2738 chloroquine resistance transporter (crt) gene, partial cds', dbxrefs=[]),
 SeqRecord(seq=Seq('TATATTTTAAGTATTATTTATTTAAGTGTATGTGTAATGAATAAAATTTTTGCT...AAA'), id='PX229099.1', name='PX229099', description='Plasmodium falciparum isolate MMN-2726 chloroquine resistance transporter (crt) gene, partial cds', dbxrefs=[]),
 SeqRecord(seq=Seq('TATA

No entanto, tenha cuidado com essa técnica, porque você recuperará uma grande quantidade de registros completos, e alguns deles terão sequências bem grandes dentro. Você corre o risco de baixar muitos dados (o que seria um esforço tanto do seu lado quanto dos servidores do NCBI)

In [ ]:
recs[0].seq

Seq('TATATTTTAAGTATTATTTATTTAAGTGTATGTGTAATGAATAAAATTTTTGCT...AAA')

In [ ]:
record = 0
for rec in recs:
  if rec.name == 'KM288867': # try finding CRT gene in 10000 records we fetched
    record = rec
    break
print(rec.name)
print(rec.description)

KM288867
Plasmodium falciparum clone PF3D7_0709000 chloroquine resistance transporter (CRT) gene, complete cds


In [ ]:
record.description

'Plasmodium falciparum clone PF3D7_0709000 chloroquine resistance transporter (CRT) gene, complete cds'

In [ ]:
str(record.seq)

'ATATGTAAAACCAAAATAAATTAAACAGAATTTATTTTTAAAAGATTTATTTGTAACAATATTACCATGATGATTTATTAAAGTAAAATCACCACCTATTAATGGTTTTCCTATACTTTCCATTGTAGTTTTTCCAAAACCTTTTTTTTTATTTTTCTCATTTTGTAATAAATATAAATATAAATATAATGCTGGTATCGTTAATGATAAATTAATACCTAAACATTTCCATGTTAAAAATATATTTTTCTTTTTCGACTTTTTTTTATTATCATTTTCATCCTCAACTTTTTTACTCATACTACTATAATCATTTCTTATAAAACTTTTATAAAATATATTTTCCCTTTTTTTATAATTTCTACTAGTATCATAAACAATATTTCTTATATTCAATAATACATTAAAATTGTTATATCCATTTTTTTTTTTATTACATATTTTGTTTAAGCTATTTAAAAGCACCTTATTCATTTTTGAAATAATGAAACAGCAAAAAAACAAAAATAACGTAAACAATTATGATGTACATAAAATATATATATATATATATATATATATATATTATTTATATATTATTCACATTTACATTTATTTATTTATATCTTCCTATTTTATACGATAAAATAAATTCGATTTAAATAAAATATACGTTAAATTAAATTCTTATATCTTAACATTTCGACATACATTTATTAATTGATTTTATTTTATTTATTTTGTTATATCCATATAAATAAACGTATCGCACATGAGTTTTTTTTTTTTTTTTTTTTTTTTTTTATATAATACTTTTTAATAGTGAAACACATACATATTTTTTACCATTTTAAATATATATATGGTGTACCATTTTGGGTTAATATATTTTTTGTAATTATATATTAATATATACATAATAAATATAAATTAATATTATATATATATATATATATATATATATAAATAATTGGAGTGTCCTATTTTGAAGCTATGTCCTTTGGAATACAGCACTTTACA